# General Trees & Parent Pointer Trees
https://opendsa-server.cs.vt.edu/ODSA/Books/CS3/html/GenTreeIntro.html

## Table of Contents
- **[General Trees](#intro)**<br>
- **[Union/Find Problem](#union)**<br>
- **[Parent Pointer Trees](#ppt)**<br>
- **[Parent Pointer Trees Implementation](#pptimp)**<br>

<a id="general"></a>

## General Trees
- many organization are hierarchical in nature
    - military, most businesses, government, etc.
- binary tree is not adequate to represent organizations that have many many subordinates at lower level
- to represent these hierarchy of many arbitrary number of children, we use general trees
- general trees are trees whose internal nodes have no fixed number of children
- the following figure depicts a general tree
<img width="400px" src="./resources/generaltrees.png">

### General Tree Definitions and Terminology
- a tree, $T$ is a finite set of one or more nodes with one special node $R$, the root
- tree may have many **subtrees** rooted at some nodes that are children of $R$
    - subtrees are arranged from left to right
- a node's **out degree** is the number of children for that node
- a **forest** is a collection of one or more trees
- each node (except for root) has precisely one parent
    - a tree with $n$ nodes must have $n-1$ edges because each node, except the root, has one edge connecting that node to its parent
    
## Implementation
- implementation of general tree is much harder compared to binary tree and is ignored

<a id="union"></a>

## The Union/Find Problem
https://opendsa-server.cs.vt.edu/ODSA/Books/CS3/html/UnionFind.html

### Find: given two nodes, are they in the same tree?
### Union: how can we efficiently merge two sets (subtrees) together?
- Union/Find is applied to Kruskal's minimum spanning tree (MST)
- what data structure can effectively help us answer these two useful questions?

<a id="ppt">

## Parent Pointer Trees
- a simple way to represent general tree 
    - for each node store only a pointer to that node's parent
    - called **parent pointer representation**
- helps us precisely solve the Union/Find problem by offering two basic operations:
    1. determine if two objects are in the same set ( the **FIND** operation)
        - follow the series of parent pointers from each node to its respective root
     
    2. merge two disjoints sets together (intersection of disjoint sets is empty)
        - disjoint sets are united (the **UNION** operation)
- this 2-step process goes by the name **UNION/FIND**

<a id="pptimp">

## Parent Pointer Tree Implementation
- represented using a single array
- index represents node and the element stored represents its parent
    - a single array is used to implement a collection of trees
- uses path compression and weighted union techniques 
    - keep the height of the joined tree to as short as possible

### Path Compression
- path compression technique can be used to create extremely shallow trees
- resets the parent of every node on the path from say $X$ to $R$ to $R$
- keeps the cost of each FIND operation very close to constant

### Weighted Union
- technique to join two sets by reducing their height
    - limits the total depth of the tree to $O(logn)$
- joins the tree with fewer nodes to the tree with more nodes
    - make the smaller tree's root point to the root of the bigger tree
- visualize weighted union here:https://opendsa-server.cs.vt.edu/ODSA/Books/CS3/html/UnionFind.html

### parent pointer tree implementation as ADT

In [1]:
#include <iostream>
#include <vector>
#include <string>
#include <algorithm>
using namespace std;

In [2]:
// general Parent-Pointer Tree implementation for UNION/FIND
class ParPtrTree {
  private:
    vector<int> nodes; //nodes vector
    vector<int> weights; // weights for weighted union
  public:
    // constructor
    ParPtrTree(size_t size) {
        nodes.resize(size); //create nodes vector
        fill(nodes.begin(), nodes.end(), -1); // each node is its own root to start
        weights.resize(size); 
        fill(weights.begin(), weights.end(), 1);// set all base weights to 1
    }
    
    // Return the root of a given node with path compression
    // recursive algorithm that makes all ancestors of the current node
    // point to the root
    int FIND(int node) {
        if (nodes[node] == -1) return node;
        nodes[node] = FIND(nodes[node]);
        return nodes[node];
    }
    
    // Merge two subtrees if they are different
    void UNION(int node1, int node2) {
        int root1 = FIND(node1);
        int root2 = FIND(node2);
        // MERGE two trees
        if (root1 != root2) {
            if (weights[root1] < weights[root2]) {
                nodes[root1] = root2;
                weights[root2] += weights[root1];
            }
            else {
                nodes[root2] = root1;
                weights[root1] += weights[root2];
            }
        }    
    }
    
    string toString() {
        string parents = "[";
        string nodes = "[";
        for (int i=0; i < this->nodes.size(); i++) {
            parents += "\t" + to_string(this->nodes[i]);
            nodes += " \t" + to_string(i); 
        }
        parents += "\t]";
        nodes += "\t]";
        return parents + "\n" + nodes;
    }
};

### Test ParPtrTree
- the following test code can be modified to test examples provided here: 
https://opendsa-server.cs.vt.edu/ODSA/Books/CS3/html/UnionFind.html

In [3]:
// 10 disjoint sets: A-J : 0->9
// A: 0, B: 1, ... J: 9
ParPtrTree ptr(10);

In [4]:
cout << ptr.toString();

[	-1	-1	-1	-1	-1	-1	-1	-1	-1	-1	]
[ 	0 	1 	2 	3 	4 	5 	6 	7 	8 	9	]

In [5]:
// union nodes (H) and (J)
ptr.UNION(7, 9);
cout << ptr.toString();

[	-1	-1	-1	-1	-1	-1	-1	-1	-1	7	]
[ 	0 	1 	2 	3 	4 	5 	6 	7 	8 	9	]

In [6]:
// union nodes (G) and (I)
ptr.UNION(6, 8);
cout << ptr.toString();

[	-1	-1	-1	-1	-1	-1	-1	-1	6	7	]
[ 	0 	1 	2 	3 	4 	5 	6 	7 	8 	9	]

In [7]:
// union nodes (A) and (J)
ptr.UNION(0, 9);
cout << ptr.toString();

[	7	-1	-1	-1	-1	-1	-1	-1	6	7	]
[ 	0 	1 	2 	3 	4 	5 	6 	7 	8 	9	]

In [8]:
ptr.UNION(1, 7);
cout << ptr.toString();

[	7	7	-1	-1	-1	-1	-1	-1	6	7	]
[ 	0 	1 	2 	3 	4 	5 	6 	7 	8 	9	]

In [9]:
ptr.UNION(6, 9);
cout << ptr.toString();

[	7	7	-1	-1	-1	-1	7	-1	6	7	]
[ 	0 	1 	2 	3 	4 	5 	6 	7 	8 	9	]